# Второй скрипт: получение и загрузка данных за прошедшие сутки. 

In [1]:
import psycopg2
import configparser
from datetime import timedelta
from datetime import datetime
import datetime
import requests
import logging
import json
import pandas as pd
import smtplib, ssl
from email.message import EmailMessage
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
import re
import os
import glob

In [2]:
config = configparser.ConfigParser()
config.read("config.ini")

#БД
HOST = config["Database"]["HOST"]
DATABASE = config["Database"]["DATABASE"]
USER  = config["Database"]["USER"]
PASSWORD  = config["Database"]["PASSWORD"]
API_URL = config["API"]["api_url"]

#Почта
mail_password = config["Email"]["mail_password"]
sender_mail = config["Email"]["sender_mail"]
receiver_email  = config["Email"]["receiver_email"]
smtp_server = config["Email"]["smtp_server"]
port = config["Email"]["port"]

## Настройка логирования

In [3]:
current_date = datetime.datetime.today().date()

# Определяем путь к папке, где находятся лог-файлы
log_directory = '/Users/79169/logs'
log_file_path = os.path.join(log_directory, f'{current_date}.log')

logging.basicConfig(level = logging.INFO, filename = log_file_path, filemode = 'w', format=\
                    '%(levelname)s: %(asctime)s : Line No. : %(lineno)d - %(message)s')

logging.info("Логирование начато.")

## Обращение к API, получение и обработка данных

In [4]:
# Запрос данных из API
def get_response(API_URL, params):
    r = requests.get(API_URL, params)
    if r.status_code == 200:
        r.json()
        logging.info(f'Соединение установлено, статус код: {r.status_code}')
    else:
        logging.info(f'connection failed with status_code {r.status_code}')
        
    return r.json()

# Обработка каждой записи в ответе 
def extract_response_data(response):
    search_arr = [] 
    
    for item in response:
        if item:
            if not item['passback_params']:
                logging.info('passback_params отсутсвует')
                continue
            else:
                part = item['passback_params'].replace('\'', '\"')
                data = json.loads(part)

                search_arr.append([
                item.get('lti_user_id', None),
                data.get('oauth_consumer_key', None),
                data.get('lis_result_sourcedid', None),
                data.get('lis_outcome_service_url', None),
                item.get('is_correct', None),
                item.get('attempt_type', None),
                item.get('created_at', None)
                ])
        else:
            continue
    return search_arr

In [5]:
# Даты для API запроса (получение данных за прошедшие сутки)
START_DATE = (datetime.datetime.today() - timedelta(days = 1)).replace(hour=0, minute=0, second=0, microsecond=0)
END_DATE = (datetime.datetime.today() - timedelta(days = 1)).replace(hour=23, minute=59, second=59, microsecond=999999)

In [6]:
# Устанавливаем соединение с базой данных
conn = psycopg2.connect(
    host = HOST,
    database = DATABASE,
    user = USER,
    password = PASSWORD)

logging.info('Подключение к БД')

cursor = conn.cursor()

# SQL-запрос для создания таблицы
cursor.execute('''
    CREATE TABLE IF NOT EXISTS lms_database (
        user_id TEXT,
        oauth_consumer_key TEXT,
        lis_result_sourcedid TEXT,
        lis_outcome_service_url TEXT,
        is_correct TEXT,
        attempt_type TEXT,
        created_at TIMESTAMPTZ
    )
''')
conn.commit()

# Параметры API запроса
params = {
    'client': 'Skillfactory',
    'client_key': 'M2MGWS',
    'start': START_DATE,
    'end': END_DATE
}

# Запрос данных из API
response =  get_response(API_URL, params)

# Обработка каждой записи
arr = extract_response_data(response)

# Параметризованный SQL-запрос для добавления данных в таблицу
for row in arr:
    cursor.execute('''
        insert into lms_database values (%s, %s, %s, %s, %s, %s, %s)
      ''', row)
    conn.commit()
logging.info('Данные загружены в БД')

## Расчет метрик, подготовка статистики

In [7]:
df = pd.DataFrame(arr)
df.columns = ['user_id', 'oauth_consumer_key', 'lis_result_sourcedid', 
              'lis_outcome_service_url', 'is_correct', 'attempt_type', 'created_at']

***1. Анализ попыток:***

In [8]:
# 1. Анализ попыток:
total_cnt = len(df)
print(f'Общее количество попыток: {total_cnt}')

Общее количество попыток: 1640


Распределение попыток:

In [9]:
# Успешные попытки:
result = df.groupby(['is_correct'])['attempt_type'].count().reset_index()
result

,is_correct,attempt_type
0,0.0,563
1,1.0,315


In [10]:
success_list = result.values.tolist()
success_list 

[[0.0, 563.0], [1.0, 315.0]]

In [11]:
success_list.insert(0, result.columns.tolist())
success_list

[['is_correct', 'attempt_type'], [0.0, 563.0], [1.0, 315.0]]

In [12]:
# Успешные попытки:
# result = df.groupby(['attempt_type', 'is_correct']).agg(attempt_count=('is_correct', 'count')).reset_index()
# result

In [13]:
#result.columns.tolist()

***2. Активность пользователей:***

In [14]:
unique_users = df['user_id'].nunique()
print(f'Количество уникальных пользователей: {unique_users}')

Количество уникальных пользователей: 105


In [15]:
# Сгруппировать по user_id и подсчитать количество записей на каждого пользователя
attempts_per_user = df.groupby('user_id').size()

mean_attempts = round(attempts_per_user.mean())
median_attempts = round(attempts_per_user.median())

print(f"Среднее количество попыток на пользователя: {mean_attempts}")
print(f"Медианное количество попыток на пользователя: {median_attempts}")

Среднее количество попыток на пользователя: 16
Медианное количество попыток на пользователя: 8


In [16]:
# Сгруппировать данные по user_id и lis_result_sourcedid
grouped = df.groupby(['user_id', 'lis_result_sourcedid'])

# Функция для вычисления успешных переходов
def calculate_transition(group):
    runs = group[group['attempt_type'] == 'run'].shape[0]
    submits_success = group[(group['attempt_type'] == 'submit') & (group['is_correct'] == True)].shape[0]
    return submits_success, runs

# Применяем функцию к каждой группе и получаем общие данные
result = grouped.apply(calculate_transition)

# Разбиваем результаты на отдельные серии для суммирования
total_submits_success = result.apply(lambda x: x[0]).sum()
total_runs = result.apply(lambda x: x[1]).sum()

# Рассчитываем общее среднее значение
overall_rate = (total_submits_success / total_runs) * 100 if total_runs > 0 else 0

print(f"Процент достижения успешного 'submit' после 'run' для всех: {round(overall_rate)}")

Процент достижения успешного 'submit' после 'run' для всех: 41


## Выгрузка статистики в Google Sheets

In [17]:
# Выгрузка статистики в Google Sheets

# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive']

# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('auto-and-deploy-d398c3c583a8.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)

spreadsheet = client.open_by_key('1GWKQpj-kJ57ex0s7TYY4-zDh_P4BMtq5UXcl-qUl6mE')

today = datetime.datetime.today().strftime("%d.%m")

sheet_name = f'Отчет за {today}'
medians_sheet = spreadsheet.add_worksheet(title=sheet_name, rows=15, cols=15)

# Данные, которые будут записаны
data = [
    ['Metric', 'Value'],
    ['Общее количество попыток', total_cnt],
    ['Количество уникальных пользователей', unique_users],
    ['Среднее количество попыток на пользователя', mean_attempts],
    ['Медианное количество попыток на пользователя', median_attempts],
    ['Процент успешных "submit" после "run"', f"{round(overall_rate)}%"]
]

title = [['Распределение попыток:']]
medians_sheet.clear()
medians_sheet.insert_rows(data, row=1)
medians_sheet.insert_rows(title, row=8)
medians_sheet.insert_rows(success_list, row=9)


# Подключение для дополнительного форматирования
service = build('sheets', 'v4', credentials=creds)

request_body = {
    "requests": [
        # Выделение полужирным 'Metric' и 'Value'
        {
            "repeatCell": {
                "range": {
                    "sheetId": medians_sheet.id,
                    "startRowIndex": 0,
                    "endRowIndex": 1,
                    "startColumnIndex": 0,
                    "endColumnIndex": 2
                },
                "cell": {
                    "userEnteredFormat": {
                        "textFormat": {
                            "bold": True
                        }
                    }
                },
                "fields": "userEnteredFormat(textFormat)"
            }
        },
        # Выделение полужирным 'Распределение попыток:'
        {
            "repeatCell": {
                "range": {
                    "sheetId": medians_sheet.id,
                    "startRowIndex": 7,
                    "endRowIndex": 8,
                    "startColumnIndex": 0,
                    "endColumnIndex": 1
                },
                "cell": {
                    "userEnteredFormat": {
                        "textFormat": {
                            "bold": True
                        }
                    }
                },
                "fields": "userEnteredFormat(textFormat)"
            }
        },
        # Право-выравнивание для колонки 'Value'
        {
            "repeatCell": {
                "range": {
                    "sheetId": medians_sheet.id,
                    "startRowIndex": 1,
                    "startColumnIndex": 1,
                    "endColumnIndex": 2
                },
                "cell": {
                    "userEnteredFormat": {
                        "horizontalAlignment": "RIGHT"
                    }
                },
                "fields": "userEnteredFormat(horizontalAlignment)"
            }
        },
        # Автоматическая ширина колонки 'Metric'
        {
            "autoResizeDimensions": {
                "dimensions": {
                    "sheetId": medians_sheet.id,
                    "dimension": "COLUMNS",
                    "startIndex": 0,
                    "endIndex": 1
                }
            }
        }
    ]
}

# Реализация запросов на обновление форматов
service.spreadsheets().batchUpdate(
    spreadsheetId='1GWKQpj-kJ57ex0s7TYY4-zDh_P4BMtq5UXcl-qUl6mE',
    body=request_body
).execute()
logging.info('Статистика выгружена в Google Sheets')

## Отправка уведомления на почту

In [18]:
# Отправка сообщения на почту

yesterday = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%d.%m")
subject = 'LMS python parser '
message = f"Скрипт успешно завершил свою работу. Отчет за {yesterday} подготовлен"

msg = EmailMessage()
msg['Subject'] = subject
msg['From'] = sender_mail
msg['To'] = receiver_email

msg.set_content(message)

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_mail, mail_password)
    server.send_message(msg=msg) 
logging.info('Уведомление о подготовке отчета отправлено')

## Удаление устаревших лог файлов

In [19]:
# Удаление устаревших лог файлов

# Поиск всех .log файлов в указанной папке
log_files = glob.glob(os.path.join(log_directory, '*.log'))

for log_file in log_files:
    date = re.findall(r'\d{4}-\d{2}-\d{2}', log_file)[0]  # Извлечение даты
    # Сравнение даты файла с сегодняшней датой минус 3 дня
    if datetime.datetime.strptime(date, '%Y-%m-%d') <= (datetime.datetime.today() - datetime.timedelta(days=3)):
        os.remove(log_file)
        logging.info(f'Удалён файл {log_file}')

# Вывод сообщения о завершении
logging.info("Удаление устаревших .log файлов завершено.")
logging.info("Логирование завершено.")